## Imports

Seuqnece Generator Imports

In [1]:
from DNA_Sequence_Generator import DNA_Sequence_Generator as DNA

In [2]:
# Generate a random DNA sequence of length 20
sequence = DNA.sequence_generator(2000, 0.2)
print(sequence)

AAATACATCGAGAGATTCGCTAACTCATATTACTTTATGATGATTTTAAGTCTAAAATAAGATTATCGATTTTAATCTAAGTATAAATTTTTTACTAAATTACAGCTAAAAATTAGACATTGTAAACTTTATTTATTCGTTATAAATGAGACAATAAAGAATTCTATTTAACCACGTTTAAAATGCATGTAACAATAATTTAAAATTAAAGTGACTATTATCCTGTATAGTTTTACTTATCACACTCTTAGAAGTTTAATATAAGTATATTAAATAAAAAATATTAAATTTTTTAAAGTAAAATTTTGATATAACTATTAGAAGATAAAATTTATTAAAATAAAACTTGTTTTCTTTATTGATATATCAATATTCAGGTTTTTTGAGGAAAGAATTAATATCGATTAAATTATTTCGGCTTAAATAATTATATTTTTATTTTTAAATTATTAGTTTTTATTGATCAACTTAATTCAACATATAGTTTAATTATAAAACACAGATATATTAATCATGTATTTAAACTCACTGATCATGTTTGATATTCATTAATATAAATAAATATCCTTTGATCAATAGTAAAATCTAATCTTATATTAAATTAACACCGTAGTTTTTTAATTAATTGCCTCTTTAAATTTTAAATGTTAATTGTTTGTATATTTTAGATATTCTTAATTAAGATACTAGTTCTTACAAAACGAATATCTATTTTATTCCGCACCAAGTTTTGAAAATAATATTTTATTATTTGTATCTTAGATCTAAAGAACAGTACACCATTATAAAAAATTTTAATTAAGCGCATTCAATACGAAATAGTATTGCGTACTTGTAGTGTATAAAACAGAACTGATTAAAATTTTAATTTACTATGTTGGAAATTGTGTGAATTTATTGGATTTCGTCTCATAACTTTTAATAATTAATAAAAATAATATGATGAAATTCTTGTTCTGTAAAATTTATGTAATACAGATTGAGTGTTCTATAAGAAATT

In [3]:
header = "example_sequence"
file_name = "example_sequence"  
DNA.sequence_to_fasta(sequence, header, file_name)